In [10]:
from __future__ import print_function, division
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py 
import plotly.graph_objs as go 
py.init_notebook_mode(connected=True)
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [2]:
edges = pd.read_csv("Dataset/elliptic_txs_edgelist.csv")
features = pd.read_csv("Dataset/elliptic_txs_features.csv",header=None)
classes = pd.read_csv("Dataset/elliptic_txs_classes.csv")

In [3]:
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
features.columns = ["txId","time_step"] + tx_features + agg_features
features = pd.merge(features,classes,left_on="txId",right_on="txId",how='left')
features['class'] = features['class'].apply(lambda x: '0' if x == "unknown" else x)

In [4]:
count_by_class = features[["time_step",'class']].groupby(['time_step','class']).size().to_frame().reset_index()
illicit_count = count_by_class[count_by_class['class'] == '1']
licit_count = count_by_class[count_by_class['class'] == '2']
unknown_count = count_by_class[count_by_class['class'] == "0"]

In [5]:
data = features[(features['class']=='1') | (features['class']=='2')]

In [6]:
X = data[tx_features+agg_features]
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

In [9]:
# Train an XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=50, max_depth=100, random_state=15)
xgb_model.fit(X_train, y_train)

# Predict on the test set and evaluate the performance
preds = xgb_model.predict(X_test)
prec, rec, f1, num = precision_recall_fscore_support(y_test, preds, average=None)
print("XGBoost Classifier- All features")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f" % (prec[1], rec[1], f1[1]))
micro_f1 = f1_score(y_test, preds, average='micro')
print("Micro-Average F1 Score:", micro_f1)

XGBoost Classifier- All features
Precision:0.982 
Recall:0.617 
F1 Score:0.758
Micro-Average F1 Score: 0.9751610594130279


In [12]:
X = data[tx_features]
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

In [15]:
# Train an XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=50, max_depth=100, random_state=15)
xgb_model.fit(X_train, y_train)

# Predict on the test set and evaluate the performance
preds = xgb_model.predict(X_test)
prec, rec, f1, num = precision_recall_fscore_support(y_test, preds, average=None)
print("XGBoost Classifier- Local features")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test,preds,average='micro')
print("Micro-Average F1 Score:",micro_f1)

XGBoost Classifier- Local features
Precision:0.929 
Recall:0.592 
F1 Score:0.723
Micro-Average F1 Score: 0.9714387974230494


In [16]:
embed_names = ["emb_"+str(i) for i in range(1,51)]
embeddings = pd.read_csv('elliptic.emb',delimiter=" ",skiprows=1,header=None)
embeddings.columns = ['txId'] + ["emb_"+str(i) for i in range(1,51)]

In [17]:
data = features[(features['class']=='1') | (features['class']=='2')]
data = pd.merge(data,embeddings,how='inner')
X = data[tx_features+agg_features+embed_names]
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

In [18]:
# Train an XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=50, max_depth=100, random_state=15)
xgb_model.fit(X_train, y_train)

# Predict on the test set and evaluate the performance
preds = xgb_model.predict(X_test)
prec, rec, f1, num = precision_recall_fscore_support(y_test, preds, average=None)
print("XGBoost Classifier - All features and node embeddings")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test,preds,average='micro')
print("Micro-Average F1 Score:",micro_f1)

XGBoost Classifier - All features and node embeddings
Precision:0.982 
Recall:0.613 
F1 Score:0.755
Micro-Average F1 Score: 0.9750053713385376


In [19]:
data = features[(features['class']=='1') | (features['class']=='2')]
data = pd.merge(data,embeddings,how='inner')
X = data[tx_features+embed_names]
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

In [20]:
# Train an XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=50, max_depth=100, random_state=15)
xgb_model.fit(X_train, y_train)

# Predict on the test set and evaluate the performance
preds = xgb_model.predict(X_test)
prec, rec, f1, num = precision_recall_fscore_support(y_test, preds, average=None)

print("XGBoost Classifier - Local features and node embeddings")
print("Precision:%.3f \nRecall:%.3f \nF1 Score:%.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test,preds,average='micro')
print("Micro-Average F1 Score:",micro_f1)

XGBoost Classifier - Local features and node embeddings
Precision:0.937 
Recall:0.598 
F1 Score:0.730
Micro-Average F1 Score: 0.972283893146172
